# Erklärung Kalibrierung (Calib)

Dieser Abschnitt erklärt die meisten Methoden der Klasse Calib. Weggelassen wurden Methoden, welche nur für die
Entwicklung benötigt wurden und im Produktivcode nicht aufgerufen werden (Debugging Methoden).

### Disclaimer
Der Code ist nicht lauffähig, dafür existieren die Python Skripte. Das Notebook stellt lediglich eine Dokumentation dar.

### Verbesserungen, welche an Calib vorgenommen wurden
- Keine spezifischen

### Import aller spezifischen Bibliotheken und Module
Zu beachten ist die Bibliothek pickle, welche zum export und import der Kalibrierung benutzt wird.

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

import pickle
import glob
import os

### Klassendeklaration und init Methode
Hier werden die Konstanten zur Bestimmung der Kalibrierungsmatrix festgelegt. Dazu gehören die Gittergröße des
Schachbrets, das Suchkriterium, den Speicherpfad und der Ordner zu den Kalibrierungsbilder. Die Init definiert dann die
restlichen Variablen. Es besteht dabei die Möglichkeit einen Pfad für eine existierende Kalibrierungsdatei festzulegen.
Ist dieser Pfad beim Erzeugen des Objekts definiert, werden die Daten von dieser Datei geladen. Wenn nicht bzw. wenn der
Pfad nicht existiert, dann wird die Kamerakalibrierung mit den Kalibrierungsbildern durchgeführt.

In [ ]:
class Calibration():
    # Chessboard Configuration
    GRID_SIZE = (9, 6)
    CRITERIA = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    SAVE_PATH = 'calibration.calib'
    CALIB_PATH = 'img/Udacity/calib'

    def __init__(self, debug: bool, file_path: str = None) -> None:
        self.debug = debug
        self.calibrated = False

        # Initialize the calibration matrix and distortion coefficients
        self.mtx, self.dist, self.roi, self.newcameramtx = None, None, None, None
        self.rvecs, self.tvecs = None, None

        # Arrays to store object points and image points from all the images.
        self.objPoints = [] # 3d point in real world space
        self.imgPoints = [] # 2d points in image plane.

        # Load calibration if file provided or the calibration already exists
        if file_path and os.path.exists(file_path): self.loadCalibration(file_path)
        elif os.path.exists(self.SAVE_PATH): self.loadCalibration(self.SAVE_PATH)

## load calibration
Diese Methode lädt unter Angabe des Dateipfads, die Kalibrierungsdaten aus der pickle Datei. Danach wird das Flag, dass
die Kalibrierung durchgeführt wurde gesetzt.

In [ ]:
def loadCalibration(self, file_path):
        with open(file_path, 'rb') as f:
            self.mtx, self.dist, self.roi, self.newcameramtx = pickle.load(f)
        self.calibrated = True

## save calibration
Diese Methode speichert die Kalibrierungsdaten in einer pickle Datei unter dem gegebenen Pfad.

In [ ]:
def saveCalibration(self, file_path):
        with open(file_path, 'wb') as f:
            pickle.dump((self.mtx, self.dist, self.roi, self.newcameramtx), f)

## equalize
Hier wird das gegebene Bild mit der Kalibrierungsmatrix angepasst. Danach ist das Bild so gestreckt, dass es der
Realität entspricht. Dieser Vorgang muss später für jedes Bild durchgeführt werden. Ist die Kamera aber noch nicht
kalibriert, dann wird sie Kalibriert (überprüfung erfolgt über das Flag, self.calibrated).

In [ ]:
def equalize(self, img):
        if not self.calibrated: self._calibrate(img)

        # undistort the image
        undisortedImage = cv.undistort(img, self.mtx, self.dist, None, self.newcameramtx)

        # crop the image
        x, y, w, h = self.roi
        undisortedImage = undisortedImage[y:y+h, x:x+w]
        return undisortedImage

## chessboard
Diese Methode versucht im Schachbrett die Eckpunkte zu finden. Dafür werden die im definierten Ordner alle Bilder
durchlaufen. Um Punkte zu finden muss im Vorfeld die Schachbrettgröße definiert werden. Mit Hilfe der gefundenen Punkten können später die
Kalibrierungsdaten erzeugt werden. Will man diesen Vorgang debuggen, so wird jedes Bild, in dem Punkte gefunden wurden
mit den Punkten dargestellt. TODO: Bild einfügen

In [ ]:
def _chessboard(self):
        # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
        objp = np.zeros((self.GRID_SIZE[0] * self.GRID_SIZE[1],3), np.float32)
        objp[:,:2] = np.mgrid[0:self.GRID_SIZE[0],0:self.GRID_SIZE[1]].T.reshape(-1,2)

        # read the chessboard imag
        images = glob.glob(self.CALIB_PATH + '/*.jpg')

        for file_name in images:
            board = cv.cvtColor(cv.imread(file_name), cv.COLOR_BGR2RGB)
            gray_board = cv.cvtColor(board, cv.COLOR_RGB2GRAY)

            # Find the chess board corners
            ret, corners = cv.findChessboardCorners(gray_board, self.GRID_SIZE, None)

            # If found, add object points, image points (after refining them)
            if ret == True:
                self.objPoints.append(objp)
                corners2 = cv.cornerSubPix(gray_board,corners, (11,11), (-1,-1), self.CRITERIA)
                self.imgPoints.append(corners)

                # Draw and display the corners
                if self.debug: 
                    cv.drawChessboardCorners(board, self.GRID_SIZE, corners2, ret)
                    print("Grid matched: " + file_name)
                    cv.imshow('img', board)
                    if cv.waitKey(100) & 0xFF == ord('q'):
                        break

        if self.debug: cv.destroyAllWindows()

## calibrate
Diese Methode ist für den Aufruf aller Kalibrierungsfunktionen verantwortlich. Zuerst werden die Punkte auf dem
Schachbrett gefunden, danach die Kalibrierungsmatrix generiert und gespeichert.

In [ ]:
def _calibrate(self, img):
        self._chessboard()
        self._generateMatrix(img)
        self.saveCalibration(self.SAVE_PATH) 

## generate matrix
Diese Methode generiert auf basis der gefundenen Fixpunkte im Schachbrett die Kalibrierungsmatrix. Wenn die Matrix
erzeugt ist, dann wird das Flag gesetzt, dass die Kamera kalibriert wurde.

In [ ]:
def _generateMatrix(self, img):
        # Generate the calibration matrix
        img_cvt = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
        ret, self.mtx, self.dist, self.rvecs, self.tvecs = cv.calibrateCamera(self.objPoints, self.imgPoints, img_cvt.shape[::-1], None, None)
        
        img = cv.imread('img/Udacity/calib/calibration3.jpg')
        h,  w = img.shape[:2]
        self.newcameramtx, self.roi = cv.getOptimalNewCameraMatrix(self.mtx, self.dist, (w,h), 1, (w,h))
        self.calibrated = True